In [40]:
offset = 0
limit = 3000
period = '7y' # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

In [41]:
import pandas as pd

data = pd.read_csv("Datasets/nasdaq_screener_1698005734880.csv", sep=',')
#data_clean = data[data['Test Issue'] == 'N']
symbols = data['Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 72


In [42]:
import yfinance as yf
import os, contextlib
import pymongo as pm

In [43]:
client = pm.MongoClient("mongodb://192.168.0.109:27017")
db = client["tarea"]
collection = db.create_collection("stocks")

In [44]:
%%time

limit = limit if limit else len(symbols)
end = min(offset + limit, len(symbols))
is_valid = [False] * len(symbols)
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            s = symbols[i]
            data = yf.download(s, period=period)
            if len(data.index) == 0:
                continue
        
            is_valid[i] = True
            #data.to_csv('Datasets/hist/{}.csv'.format(s))
            df = data.reset_index()
            df["Symbol"] = s
            collection.insert_many( df.to_dict('records') )
            
print('Total number of valid symbols downloaded = {}'.format(sum(is_valid)))

Total number of valid symbols downloaded = 72
CPU times: total: 12.7 s
Wall time: 29.5 s
